# Initialize database tables

## Control table

In [38]:
import os
import numpy as np

from grizli.aws import db
from grizli import utils

import djapipes

In [4]:
djapipes.run.run_pipes_on_dja_spec?

Signature:
djapipes.run.run_pipes_on_dja_spec(
    spec_name,
    sfh='continuity',
    n_age_bins=10,
    scale_disp=1.3,
    dust_type='kriek',
    use_msa_resamp=False,
    fit_agn=False,
    fit_dla=False,
    fit_mlpoly=False,
)
Docstring:
Runs bagpipes on spectrum from DJA AWS database, saves posteriors as .h5 files and plots as .pdf files

Parameters
----------
spec_name : DJA spectrum name, format=str

Returns
-------
None
File:      ~/miniconda3/envs/py310-djapipes/lib/python3.10/site-packages/djapipes/run.py
Type:      function

In [31]:
import time
row = dict(
    file_spec = "rubies-egs61-v3_prism-clear_4233_42328.spec.fits",
    sfh='continuity',
    n_age_bins=10,
    scale_disp=1.3,
    dust_type='kriek',
    use_msa_resamp=False,
    fit_agn=False,
    fit_dla=False,
    fit_mlpoly=False,
    status=70,
    ctime=time.time(),
)

tab = utils.GTable([row])
tab

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


In [33]:
if (0):
    # db.execute("DROP TABLE nirspec_bagpipes")
    db.send_to_database("nirspec_bagpipes", tab, if_exists="replace")
    db.execute("CREATE INDEX on nirspec_bagpipes (file_spec)")


db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


## Results table

In [45]:
# ! ls pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/
post = utils.read_catalog("pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv")
post["sfh"] = "continuity"
post["dust_type"] = "kriek"
for c in post.colnames:
    if post[c].dtype == np.float64:
        post[c] = post[c].astype(np.float32)

if (0):
    db.send_to_database("nirspec_bagpipes_posterior", post, if_exists="replace")
    db.execute("CREATE INDEX on nirspec_bagpipes_posterior (file_spec)")

db.SQL("select * from nirspec_bagpipes_posterior")

file_spec,file_phot,file_zout,id_phot,dr,filt_num,calib_0_16,calib_0_50,calib_0_84,calib_1_16,calib_1_50,calib_1_84,calib_2_16,calib_2_50,calib_2_84,continuity_dsfr1_16,continuity_dsfr1_50,continuity_dsfr1_84,continuity_dsfr2_16,continuity_dsfr2_50,continuity_dsfr2_84,continuity_dsfr3_16,continuity_dsfr3_50,continuity_dsfr3_84,continuity_dsfr4_16,continuity_dsfr4_50,continuity_dsfr4_84,continuity_dsfr5_16,continuity_dsfr5_50,continuity_dsfr5_84,continuity_dsfr6_16,continuity_dsfr6_50,continuity_dsfr6_84,continuity_dsfr7_16,continuity_dsfr7_50,continuity_dsfr7_84,continuity_dsfr8_16,continuity_dsfr8_50,continuity_dsfr8_84,continuity_massformed_16,continuity_massformed_50,continuity_massformed_84,continuity_metallicity_16,continuity_metallicity_50,continuity_metallicity_84,dust_Av_16,dust_Av_50,dust_Av_84,nebular_logU_16,nebular_logU_50,nebular_logU_84,noise_scaling_16,noise_scaling_50,noise_scaling_84,redshift_16,redshift_50,redshift_84,stellar_mass_16,stellar_mass_50,stellar_mass_84,formed_mass_16,formed_mass_50,formed_mass_84,sfr_16,sfr_50,sfr_84,ssfr_16,ssfr_50,ssfr_84,nsfr_16,nsfr_50,nsfr_84,mass_weighted_age_16,mass_weighted_age_50,mass_weighted_age_84,tform_16,tform_50,tform_84,tquench_16,tquench_50,tquench_84,mass_weighted_zmet_16,mass_weighted_zmet_50,mass_weighted_zmet_84,sfh_16,sfh_50,sfh_84,chisq_phot_16,chisq_phot_50,chisq_phot_84,restU_16,restU_50,restU_84,restV_16,restV_50,restV_84,restJ_16,restJ_50,restJ_84,sfh,dust_type
str48,str36,str43,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str10,str5
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,ceers-full-grizli-v7.4-fix_phot.fits,ceers-full-grizli-v7.4-fix.eazypy.zout.fits,45984,0.0025837352,7,1.0906843,1.1007513,1.111982,0.13404463,0.14948104,0.16724166,-0.025392357,-0.01450396,-0.005704524,1.5992963,2.3499546,3.897448,-5.432896,-2.759181,-1.5772108,-2.6653802,-0.920001,0.101954564,-2.2580905,-0.7151477,0.3022016,-1.9315832,-0.53638446,0.43751964,-1.5627255,-0.2930139,0.77155346,-1.3798062,-0.2493546,0.7266544,-1.347817,-0.22493288,0.79783887,10.594977,10.6036625,10.613153,1.9617285,2.0898528,2.2323523,0.26661077,0.31436917,0.3580495,-3.4096978,-2.4284744,-1.4446774,1.6471881,1.7081825,1.7717766,2.6793857,2.6820676,2.6856956,10.358072,10.366877,10.377017,10.594977,10.6036625,10.613153,3.2800497e-08,3.7763068e-05,0.003439571,-17.849339,-14.788182,-12.8288145,-8.708921,-5.647402,-3.6870823,1.2075636,1.2132057,1.2291875,1.158053,1.1736107,1.1806157,1.4938755,1.497463,1.5014544,1.9557912,2.0847487,2.228576,0.0,1.2216088e-05,0.032659084,13.353081,14.23861,15.93904,4.925325,4.945362,4.966345,2.97721,2.9887047,3.0007243,1.7958202,1.8099265,1.8247521,continuity,kriek
